<img src="assets/micropython-esp32.png">


***
# MicroPython on €10 hardware


[MicroPython](https://micropython.org/) is an implementation of the Python 3 optimised to run on microcontrollers.
- MicroPython started as a Kickstarter campaign in 2013 by Damien George.
- Designed to run on constrained environments. 128K ROM/8K RAM is the recommended minimum configuration.
- Compilation is on the chip so no need to install tools on your laptop.
- Can interact with the the REPL to run code directly on the hardware. 
- Includes a subset of core Python libraries (most beginning with 'u'). Hardware specific libraries that vary for each hardware port.
- If you don't have a microcontroller try out the [online emulator](http://micropython.org/unicorn/)



MicroPython downloads and ESP32 installation instructions are available [here](https://micropython.org/download#esp32).


***
# Jupyter Notebook MicroPython kernel install

We are going to be using the [Jupyter MicroPython Remote kernel](https://pypi.org/project/jupyter-micropython-remote/) to interact with a MicroPython board over its REPL interface. 



From the shell command to install the MicroPython kernel for Jupyter run:

```
pip install jupyter_micropython_remote
```

Next register the kernel with Jupyter:

```
python -m mpy_kernel.install
```
    
And that should be it!!! Run Jupyter notebooks or Jupyter labs:

```
jupyter notebook
```
or
```
jupyter lab
```

You should now have a **MicroPython Remote** option when creating a new Jupyter notebook.



***
# What does €10 get you?


<img src="assets/esp32_5_10_15.png">


Quite a lot actually:
- €5  ESP32 with 4MB flash, Wi-Fi & Bluetooth, lots of I/O.
- €10 ESP32 with 16MB flash, 8MB PSRAM, Wi-Fi & Bluetooth, lots of I/O, lithium battery charging circuit, SD card slot.
- €15 ESP32 with 4MB flash, Wi-Fi & Bluetooth, lots of I/O, lithium battery charging circuit, OLED display, LoRa wireless with km range.

ESP32 developer boards typically consist of:
- ESP32 System On Chip (SOC) as seen at top of the first two of these boards.
- USB interface for power and primary communication during development as seen at the bottom of these boards.
- GPIO and power pins exposes on two rows of pins either side of the board.
- Various manufacturers provide additional features on their boards like battery charging, display, camera, LoRa wireless, GPS.


<img src="assets/esp32-block-diagram.jpg">



The ESP32 SOC:
- Dual core, clock frequency up to 240MHz, 512 KB internal RAM.
- Modules come in different variants with additional external flash and PSRAM.
- Runs 32 bit programs.
- Wi-Fi and bluetooth built-in.
- Up to 34 GPIO pins.
- Ultra low power co-processor with access to GPIO during deepsleep.
- Built in security and encrytion hardware.



Check available documentation before buying. 
- Are board schematics available (usually based on an Espressif reference design)? 
- Are common parts used for display, LoRa chips, etc? There is better library support for common components.


***
# Establish a notebook connection to your microcontroller 

The first cell of your Jupyter Notebook should contain something like this:

```
%connect <device> --baudrate=115200 --user='micro' --password='python' --wait=0
```

You may need to reset the microcontroller with a soft reboot:

```
%reboot
```

**Note: Remember that you should only have one connection open on the serial port connected to the hardware.**



In [1]:
%connect COM4 --baudrate=115200

Connected on COM4


In [ ]:
%disconnect

In [1]:
# Lets look at some of the microcontroller system details

import sys

print('Platform: {}'.format(sys.platform))
print('MicroPython Version: {}\n'.format(sys.implementation))


Platform: esp32
MicroPython Version: (name='micropython', version=(1, 13, 0), mpy=10757)



***
# MicroPython modules

Lets see what MicroPython libraries are available on the ESP32 board:

- [MicroPython standard libraries and micro-libraries](https://docs.micropython.org/en/latest/library/index.html#python-standard-libraries-and-micro-libraries)




In [ ]:
# Get a list of modules available for the ESP32 port.

help('modules')

In [ ]:
# The machine module contains functions related to the hardware on the board. 
# Including: I2C, SPI, UART interface, etc

import machine

help(machine)

***
# Controlling machine hardware

Lets work through some examples of controlling hardware with MicroPython.


<img src="assets/esp32_pinout.png">


Peripheral Input/Output include:
- 2 × I²C (Inter-Integrated Circuit).
- 3 x UART (universal asynchronous receiver/transmitter).
- 4 × SPI (Serial Peripheral Interface).
- 2 × I²S (Integrated Inter-IC Sound).
- CAN 2.0 (Controller Area Network).
- PWM (pulse width modulation) up to 16 channels. Useful for LED or motor control.
- 10 x Capacitive touch sensors.
- 12-bit ADCs (analog-to-digital converter) up to 18 channels.
- 2 × 8 bit DACs (digital-to-analog converter).
- Hall sensor.
- Internal temperature sensor.



***
## Blink the 'Hello World' for microcontroller

Lets set a [GPIO pin](https://docs.micropython.org/en/latest/library/machine.Pin.html) as an output and use it to drive an LED. 


In [1]:
# Set GPIO pin as output and toggle state.

from machine import Pin
import time

led = Pin(18, Pin.OUT)
led.off()

print('Blink LED', end= ' ')

for _ in range(10):
    led.value(not led.value())
    time.sleep_ms(1000)
    print('.', end= ' ')
    


Blink LED . . . . . . . . . . 

***
## Pulse Width Modulation (PWM)

PWM is typically used to control the brightness of a LED or to control motor speed.

To use PWM first create a Pin object and then specify the pulse frequency and duty cycle.
- ESP32 can generate PWM on most I/O pins.
- Frequency is from 1Hz to several MHz.
- Duty cycle is between 0 (all off) and 1023 (all on), with 512 being 50% duty.

<img src="assets/pwm.jpg" width="700">


In [ ]:
# Demo: Controller LED with PWM.

from machine import Pin, PWM

led_pwm = PWM(Pin(18), freq=2, duty=512)

print('Frequency: {} Hz'.format(led_pwm.freq()))
print('Duty Cycle: {:.1%}'.format(led_pwm.duty()/1024.0))

In [ ]:
# Deactivate PWM

led_pwm.deinit()

***
## Hardware timer

To use a [hardware timer](https://docs.micropython.org/en/latest/library/machine.Timer.html) select a timer id, the trigger mode, and specify a callback function.
- The ESP32 has 4 hardware timers. Can also specify a software timer by using id = -1
- Timer period in ms.
- Timer mode:
 - ```Timer.ONE_SHOT``` - The timer runs once until the configured period of the channel expires.
 - ```Timer.PERIODIC``` - The timer runs periodically at the configured frequency of the channel.
- Callback function is called when triggered.



In [ ]:
# Control a LED with a Timer.

from machine import Pin, Timer

led = Pin(18, Pin.OUT)

def toggle_led(timer):
    led.value(not led.value())
    
# Use hardware timer 0
led_timer = Timer(0)
led_timer.init(period=1000, mode=Timer.PERIODIC, callback=toggle_led)


In [ ]:
# Disable timer.

led_timer.deinit()

***
## Hardware interrupt

A [hardware interrupt](https://docs.micropython.org/en/latest/library/machine.Pin.html) occurs in response to an external event, eg. a GPIO Interrupt (when a key is pressed).

- You can use all GPIOs as interrupts, except GPIO 6 to GPIO 11 (reserved for external flash memory interface).
- Set a callback function to be executed on the trigger. Callback functions should be as short and simple as possible.
- Interrupts can be triggers on rising/falling edge or pin level.


In [ ]:
# Trigger a hardware interrupt from a button press and toggle a LED.
# LEDs: Pin 18
# Button: Pin 0

from machine import Pin

led = Pin(18, Pin.OUT)
button = Pin(0, mode=Pin.IN, pull=Pin.PULL_UP)

def toggle_led(pin):
    led.value(not led.value())

button.irq(handler=toggle_led, trigger=Pin.IRQ_FALLING)


In [ ]:
# Disable IRQ

button.irq(trigger=0)

***
## Capacitive touch

There are ten [capacitive touch](http://docs.micropython.org/en/latest/esp32/quickref.html#capacitive-touch) enabled pins that can be used on the ESP32: 0, 2, 4, 12, 13 14, 15, 27, 32, 33.

For the demonstration circuit we are using active low logic, ie. the led will turn on when the output pin logic level is 0 (low). The [Signal class](https://docs.micropython.org/en/latest/library/machine.Signal.html) is an extension of the Pin class which can be in “asserted” (on) or “deasserted” (off) states, while being inverted (active-low) or not.

<img src="assets/active_low_led.png">




In [ ]:
from machine import TouchPad, Pin, Signal
import utime

threshold = 300

# The signal class allows to abstract away active-high/active-low difference
ledR = Signal(18, Pin.OUT, invert=True)

touch = TouchPad(Pin(2))

while True:
    if touch.read() < threshold:
        ledR.on()
    else:
        ledR.off()
    utime.sleep_ms(100)
    

***

# I²C bus & sensors

<img src="assets/I2C_bus.png">

The ESP32 offers support for a wide variety of peripheral interfaces with one of the most popular being Inter-Integrated Circuit (I²C or I2C). I²C  uses two wires to communicate between the bus master and slave devices: a bi-directional serial data wire (SDA), and clock line (SCL). The bus supports multiple masters and up to 112 slaves using the 7 bit address scheme.

I²C is a popular interface for slower devices (Standard mode (100 Kbit/s), Fast mode (400 Kbit/s)) as it is cheap and requires minimum wiring for the device (typically VCC, GND, SDA, SCL).




***
## I²C bus scan

Each I²C device has a unique address. A bus scan cycles through all 127 possible device addresses and checks whether or not an acknowledge is received. 

A useful list of common I²C device addresses can be found [here](https://learn.adafruit.com/i2c-addresses/the-list).



In [1]:
# Scan the I2C bus for devices

from machine import I2C, Pin

i2c = I2C(0, scl=Pin(22), sda=Pin(21))

print('Scan i2c bus...')
devices = i2c.scan()

if len(devices) == 0:
    print('No i2c device !')
else:
    print('i2c devices found:', len(devices))
    for device in devices:
        print('Decimal address: ', device, ' | Hex address: ', hex(device))


Scan i2c bus...
i2c devices found: 2
Decimal address:  104  | Hex address:  0x68
Decimal address:  118  | Hex address:  0x76


****

## BME280 environmental sensor

<img src="assets/bme280.jpg">

The BME280 sensor measures temperature, humidity, and barometric pressure.
- I²C address: 0x76 (default) or 0x77
- [BME280 tutorial with MicroPython library](https://randomnerdtutorials.com/micropython-bme280-esp32-esp8266/). 

I have already uploaded the Micropython BME280 module to the board flash in directory ```/lib```. MicroPython looks for modules using ```sys.path``` which defaults to ```['', '/lib']```



In [ ]:
# Attach a BME280 sensor via I2C bus and read temperature, pressure & humidity

from machine import Pin, I2C
from time import sleep
import BME280

# ESP32 - Pin assignment
i2c = I2C(0, scl=Pin(22), sda=Pin(21), freq=10000)
bme = BME280.BME280(i2c=i2c, address=0x76)

for _ in range(3):
    print('Temperature: ', bme.temperature)
    print('Humidity: ', bme.humidity)
    print('Pressure: ', bme.pressure)
    sleep(3)


***
## Inertial Measurement Unit


<img src="assets/RollPitchYaw.jpg">


The MPU6050 is a 6 DOF (degrees of freedom) IMU (inertial measurement unit) sensor with the 3 accelerometer outputs and the 3 gyroscope outputs:
- I²C address: 0x68
- Demo is based on [MPU6050 example](https://github.com/nihalpasham/micropython_sensorfusion)




In [ ]:
# Attach MPU6050 and using a Kalman fusion filter to convert acceleration and angular velocity to roll and pitch measurements

import imu
import utime

imu.init_MPU()
imu.calibrate_sensors()

while True:
    x, y = imu.read_imu6050()
    print('Kalman Filter Roll: {:2.2f}  Pitch: {:2.2f}     '.format(x,y), end='\r')
    utime.sleep_ms(250)


In [ ]:
%connect COM4 --baudrate=115200

In [ ]:
%disconnect

***
# Wireless connectivity

The ESP32 offers a cheap solution for IoT projects requiring WiFi and Bluetooth capability. Wireless protocols that are supported:
- [Wi-Fi](https://docs.micropython.org/en/latest/library/network.WLAN.html): 802.11 b/g/n.
- [Bluetooth](https://docs.micropython.org/en/latest/library/ubluetooth.html): v5.0 and BLE.


Can configure an ESP32 as a WiFi Station ```network.STA_IF``` or Access Point ```network.AP_IF```.


In [ ]:
# Scan for available WiFi networks.

import network

wlan = network.WLAN(network.STA_IF)
wlan.active(True)

networks = wlan.scan()
for ssid, bssid, channel, rssi, authmode, hidden in sorted(networks, key=lambda x: x[3], reverse=True):
    print('ssid: {}, bssid: {}, channel: {}, RSSI: {}, authmode: {}, hidden: {}'.format(ssid, bssid, channel, rssi, authmode, hidden))

wlan.disconnect()

***
MicroPython recently added multicast DNS (mDNS) support for the ESP32 so you can now assign a DHCP hostname for the device and access it by name instead of needing to know its IP address when on the local subnet.


In [ ]:
# Connect ESP32 to local WiFi network using credentials read from a configuration file. Set DHCP hostname.

from config import wifi_config
import network

wlan = network.WLAN(network.STA_IF)

if not wlan.isconnected():
    print('connecting to network...')
    wlan.active(True)
    # It is important to set the hostname parameter before the network connection is established.
    wlan.config(dhcp_hostname = 'learning_week')
    wlan.connect(wifi_config['ssid'], wifi_config['password'])
    while not wlan.isconnected():
        pass

print('Network config: ', wlan.ifconfig())
print('DHCP hostname: ', wlan.config('dhcp_hostname'))


***
## Set real time clock from NTP server

The ESP32 has an internal [RTC](http://docs.micropython.org/en/latest/library/machine.RTC.html) however it is not very accurate. If you need accurate time measurement over an extended period use a temperature compensated RTC like the DS3231.

Since we have wireless conectivity we can sychronise the ESP32 clock with an internet NTP server.



In [ ]:
# Connect to NTP server and set RTC

import ntptime, machine

rtc = machine.RTC()
print('Before NTP sync: {}'.format(rtc.datetime()))

#ntptime.host = 'pool.ntp.org'    # Default host used if not set
ntptime.host = 'time.google.com'

ntptime.settime()

print('After NTP sync:  {}'.format(rtc.datetime()))


***
## HTTP requests

Micropython HTTP library is ```urequests``` and has support for:
- SSL
- Cookies
- Basic Auth
- Custom HTTP Headers
- GET, POST, PUT, PATCH, DELETE, HEAD




In [ ]:
import urequests

help(urequests)


In [ ]:
# Get a list of Github API endpoints

import urequests

#url = 'http://jsonplaceholder.typicode.com/todos/22'
url = 'https://api.github.com'
headers = {'User-agent': 'micropython-urequests/1.13.0', 'Accept': 'application/json'}

response = urequests.get(url, headers = headers)

#print('\nResponse content: {}'.format(response.content))
# Since the response is serialized JSON, we can deserialise into a dictionary for easier parsing
print('\nResponse as dict: {}'.format(response.json()))
print('\nStatus code: {}'.format(response.status_code))
print('\nReason: {}'.format(response.reason.decode('utf-8')))
    
response.close()


***
# Concurrent tasks with AsyncIO

Concurrent programming is provided in Micropython with the [uasyncio](https://docs.micropython.org/en/latest/library/uasyncio.html) library. It implements a subset of the corresponding CPython module.

Coroutines are declared with the ```async/await``` syntax in asyncio applications. Use ```create_task()``` to schedule the execution of a coroutine object, followed by ```asyncio.run()```.

A good tutorial on the use of ```asycnio``` on embedded hardware can be found [here](https://github.com/peterhinch/micropython-async/blob/master/v3/docs/TUTORIAL.md).


In [ ]:
print('IP address: ', wlan.ifconfig()[0])
print('DHCP hostname: ', wlan.config('dhcp_hostname'))


In [ ]:
# Invoke garbage collection

import gc

gc.collect()


***

The following example creates two instances of the blink led task and an instance of a socket server running concurrently.

In [ ]:
from machine import Pin, I2C
import uasyncio as asyncio
import utime
import json
import BME280

EPOCH_OFFSET = 946684800   # ESP32 epoch is 2000-1-1 and Unix is 1970-1-1. Adding 946684800 (30 years)

ledR = Pin(18, Pin.OUT)
ledB = Pin(19, Pin.OUT)

async def blink(led, period_ms):
    while True:
        led.value(not led.value())
        await asyncio.sleep_ms(period_ms)


i2c = I2C(0, scl=Pin(22), sda=Pin(21), freq=10000)
bme = BME280.BME280(i2c=i2c, address=0x76)

def read_bme280():
    try:
        temperature = bme.temperature
        humidity = bme.humidity
        pressure = bme.pressure
        return(temperature, humidity, pressure)
    except:
        return(0, 0, 0)


async def serve_data(reader, writer):
    print(await reader.read(512))

    header = b'HTTP/1.1 200 OK\r\n'
    header += b'Content-Type: application/json\r\n'
    header += b'\r\n'

    data = {
        'device_ip': wlan.ifconfig()[0],
        'hostname': wlan.config('dhcp_hostname'),
        'device_time': utime.time() + EPOCH_OFFSET,
        'led_red': ledR.value(),
        'led_blue': ledB.value(),
        'temperature': read_bme280()[0],
        'humidity': read_bme280()[1],
        'pressure': read_bme280()[2]
    }

    body = json.dumps(data).encode('utf-8')
    await writer.awrite(header + body)
    await writer.aclose()


loop = asyncio.get_event_loop()
taskRed = loop.create_task(blink(ledR, 700))
taskBlue = loop.create_task(blink(ledB, 400))
# host = '0.0.0.0'. Tells the server to host on all IP addresses on all interfaces
taskServe = loop.create_task(asyncio.start_server(serve_data, '0.0.0.0', 8081))

try:
    loop.run_forever()
except KeyboardInterrupt:
    print('Keyboard interrupt....')    
finally:
    print('Closing event loop....')
    loop.close()



***

# ESP32 deep sleep

The microcontroller can enter a sleep state which stops execution in an attempt to reduce power consumption.
- ```machine.lightsleep```: In light sleep mode, digital peripherals, most of the RAM, and CPU are clock-gated, and supply voltage is reduced. Upon exit from light sleep, peripherals and CPU resume operation, their internal state is preserved.
- ```machine.deepsleep```: In deep sleep mode, CPU, most of the RAM, and all the digital peripherals are powered off. The RTC module is the only parts of the chip which is powered. Deepsleep will not retain RAM or any other state of the system (however data can be stored in the RTC module memory). Upon wake, execution is resumed from the main script, similar to a hard or power-on reset.

<img src="assets/esp32-deepsleep.jpg">

ESP32 can wake up after sleeping for a specified time. If no time interval is specified, sleep can last indefinitely or until a **wake interrupt** is received from one of the following sources:
- Timer.
- External wake up when a change in the state of a pin occurs.
- Touch pins.


In [ ]:
import machine
import esp32
from machine import Pin
from time import sleep

wake1 = Pin(0, mode = Pin.IN)

esp32.wake_on_ext1(pins = [wake1], level = esp32.WAKEUP_ALL_LOW)

for i in reversed(range(5)):
    print('Going to sleep in {} seconds...'.format(i+1))
    sleep(1)

print('Going to sleep now')


# Deep sleep until external wake up pin pressed
machine.deepsleep()


# To get the reason for the wake event use machine.wake_reason()
# Possible wakeup_reason values:
#    0 - no wake-up reason
#    1 - EXT_0 wake-up
#    2 - EXT_1 wake-up
#    3 - Touchpad wake-up
#    4 - RTC wake-up
#    5 - ULP wake-up

machine.wake_reason()


In [ ]:
# Try to reconnect after waking from deepsleep

%connect COM4 --baudrate=115200

In [ ]:
%disconnect

***
# Uploading your Python script to the board

With MicroPython boards you can add as many scripts as you like. You are only limited by the size of available flash memory. The file system contains two important files:
1. ```/boot.py``` is run when the board is powered on or reset. Generally it is not edited and should only be used to setup board configuration.
2. ```/main.py``` is an optional file and is run after boot.py. This file can be used to start your application script after power on. 



***
## rshell

[Remote MicroPython shell: rshell](https://github.com/dhylands/rshell)


To connect rshell to the MicroPytohn hardware:

```
rshell -p COM4
```

The should establish a connection to the board. rshell assumes your board is called **/pyboard/**

```
ls /pyboard -l
```

To enter the REPL on the hardware type **repl**. **help()** to display REPL help. Ctrl + X to exit the REPL.

To copy file to the ESP32 from your local directory:

```
cp *.py /pyboard
ls /pyboard
```



In [ ]:
# The os module contains functions for filesystem access.

import os

help(os)

In [ ]:

print('\n'.join(os.listdir('/')))


In [ ]:
# Disconnect notebook from board before using rshell

%disconnect

***
## VS Code & Pymakr

A recommendation on a MicroPython IDE is VS Code with the [Pymakr extension](https://docs.pycom.io/gettingstarted/software/vscode/). The Pymakr extension features:
- Connects to remote MicroPython board.
- Interact with the board with the command line REPL.
- Upload your project to the board.
- Download files from the board.
- Run current editor file or select lines of code.



***
# Debugging with Sigrok PulseView & logic analyzers

<img src="assets/LogicAnalyzer.png">


You may find it necessary to debug your hardware projects and the combination of [Sigrok PulseView](https://sigrok.org/doc/pulseview/0.4.2/manual.html) and cheap USB logic analyzer hardware (€10 to €20) is a great addition to your toolkit.

One of the nice features of PulseView is [protocol decoders](https://sigrok.org/wiki/Protocol_decoders) which can be stacked to decode signals and protocols. A broad range of decoders are supported and you can write your own custom decoder using Python 3.


<img src="assets/Sigrok_Scan_I2C_1.png">


The above PulseView trace shows a portion of an I²C bus scan with stacked I²C protocol and timing decoders.

